In [2]:
import numpy as np
import tensorflow as tf
import random
import tensorflow as tf

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.imdb.load_data(
    path = 'imdb.npz',
    num_words = None,
    skip_top = 0,
    maxlen = None,
    seed = 113,
    start_char = 1,
    oov_char = 2,
    index_from = 3)

positive_reviews = []
negative_reviews = []
for i in range(0, len(x_train)):
    if y_train[i] == 1:
        positive_reviews.append(x_train[i])
    else:
        negative_reviews.append(x_train[i])

In [3]:
selected_positive_reviews = random.sample(positive_reviews, np.int(len(positive_reviews) / 2))
balanced_positive_reviews = np.tile(selected_positive_reviews, 2)

In [4]:
word_index = tf.keras.datasets.imdb.get_word_index(path='imdb_word_index.json')
word_index = {k:(v+3) for k,v in word_index.items()}
word_index["<PAD>"] = 0
word_index["<START>"] = 1
word_index["<UNK>"] = 2
word_index["<UNUSED>"] = 3

reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])

def decode_review(text):
    return ' '.join([reverse_word_index.get(i, '?') for i in text])

master_holder =[[]]

for record in selected_positive_reviews:
    holder = []
    for idx in record:
        if idx in reverse_word_index:
            holder.append(reverse_word_index[idx])
        else:
            holder.append('UNK')
    holder.pop(0)
    master_holder.append(holder)
master_holder.pop(0)

master_list = []
master_list = [(' '.join(master_holder[idx])) for idx in range(len(master_holder))]

len(master_list)

6250

In [5]:
text = ' '.join(master_list)

In [6]:
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

96 unique characters


In [7]:
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

In [8]:
print('{')
for char,_ in zip(char2idx, range(20)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))
print('  ...\n}')

{
  ' ' :   0,
  "'" :   1,
  '0' :   2,
  '1' :   3,
  '2' :   4,
  '3' :   5,
  '4' :   6,
  '5' :   7,
  '6' :   8,
  '7' :   9,
  '8' :  10,
  '9' :  11,
  'a' :  12,
  'b' :  13,
  'c' :  14,
  'd' :  15,
  'e' :  16,
  'f' :  17,
  'g' :  18,
  'h' :  19,
  ...
}


In [9]:
# Show how the first 13 characters from the text are mapped to integers
print ('{} ---- characters mapped to int ---- > {}'.format(repr(text[:13]), text_as_int[:13]))

'i saw this mo' ---- characters mapped to int ---- > [20  0 30 12 34  0 31 19 20 30  0 24 26]


In [10]:
import numpy as np
import os
import time

seq_length = 100
examples_per_epoch = len(text)//seq_length

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in char_dataset.take(5):
    print(idx2char[i.numpy()])

i
 
s
a
w


In [11]:
sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

'i saw this movie and the play and i have to add that this was the most touching story that i had ever'
' seen until i saw this movie i was unaware of how awful life was and probably still is for the south '
'african children and adults that were and are living in that era it brought tears to my eyes and much'
' sadness to my heart that any human being should have to struggle like that just to stay alive and to'
' bring the children right out of that area and teach them to act and preform and turn them loose to t'


In [12]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'i saw this movie and the play and i have to add that this was the most touching story that i had eve'
Target data: ' saw this movie and the play and i have to add that this was the most touching story that i had ever'


In [13]:
for i, (input_idx, target_idx) in enumerate(zip(input_example[:5], target_example[:5])):
    print("Step {:4d}".format(i))
    print("  input: {} ({:s})".format(input_idx, repr(idx2char[input_idx])))
    print("  expected output: {} ({:s})".format(target_idx, repr(idx2char[target_idx])))

Step    0
  input: 20 ('i')
  expected output: 0 (' ')
Step    1
  input: 0 (' ')
  expected output: 30 ('s')
Step    2
  input: 30 ('s')
  expected output: 12 ('a')
Step    3
  input: 12 ('a')
  expected output: 34 ('w')
Step    4
  input: 34 ('w')
  expected output: 0 (' ')


In [14]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [15]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
      ])
    return model

model = build_model(
  vocab_size = len(vocab),
  embedding_dim=embedding_dim,
  rnn_units=rnn_units,
  batch_size=BATCH_SIZE)

In [16]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")


(64, 100, 96) # (batch_size, sequence_length, vocab_size)


In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           24576     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 96)            98400     
Total params: 4,061,280
Trainable params: 4,061,280
Non-trainable params: 0
_________________________________________________________________


In [18]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()
sampled_indices

array([57, 55, 41,  3, 95, 86, 51,  2, 45, 40, 53, 15, 14, 28, 70, 82, 49,
       51, 19, 39, 26, 94,  3, 55,  0, 40, 36, 37, 68, 39, 72, 15, 63, 69,
       84, 64, 71, 83, 32, 84,  9, 43, 93, 50, 14, 79, 16, 72,  7,  7, 88,
       68, 87, 44, 49, 32, 26, 49, 71, 88, 77, 10,  4, 47, 23,  8, 64, 12,
       69, 66, 23,  5, 68,  8, 41, 42, 90, 81, 84, 13,  1, 51, 93, 16,  8,
       70, 28, 15, 15, 39, 66,  4, 53, 25, 90, 24, 45, 26, 22, 57],
      dtype=int64)

In [19]:
print("Input: \n", repr("".join(idx2char[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr("".join(idx2char[sampled_indices ])))

Input: 
 'e great investigatory agency it deals with airplane bomb plots killing off of indians in oklahoma fo'

Next Char Predictions: 
 "´\xad\x8e1…ø¦0\x97\x85¨dcqäñ¡¦h\x84o”1\xad \x85yzâ\x84ædÉãôÊåóuô7\x95“£cíeæ55üâú\x96¡uo¡åüë82\x9el6Êaãàl3â6\x8e\x91–ïôb'¦“e6äqdd\x84à2¨n–m\x97ok´"


In [21]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss  = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 96)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       4.5642085


In [22]:
model.compile(optimizer='adam', loss=loss)

In [23]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

In [25]:
EPOCHS=2
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/2
1267/1267 [==============================] - 5669s 4s/step - loss: 1.6574
Epoch 2/2
1267/1267 [==============================] - 5753s 5s/step - loss: 1.3010


In [26]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints\\ckpt_2'

In [27]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [28]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            24576     
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 96)             98400     
Total params: 4,061,280
Trainable params: 4,061,280
Non-trainable params: 0
_________________________________________________________________


In [29]:
def generate_text(model, start_string):
  # Evaluation step (generating text using the learned model)

  # Number of characters to generate
    num_generate = 1000

  # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

  # Empty string to store our results
    text_generated = []

  # Low temperatures results in more predictable text.
  # Higher temperatures results in more surprising text.
  # Experiment to find the best setting.
    temperature = 1.0

  # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
      # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

      # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

      # We pass the predicted character as the next input to the model
      # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [45]:
import re

frequency = { }
match_pattern = re.findall(r'\b[a-z0-9]+\b', text)
for word in match_pattern:
    count = frequency.get(word,0)
    frequency[word] = count + 1
frequency_sorted = sorted(frequency, key=frequency.get, reverse=True)
frequency_seed = frequency_sorted[:10]
frequency_seed

['the', 'and', 'a', 'of', 'to', 'is', 'br', 'in', 'it', 'i']

In [48]:
import pickle
file_index = 0
for seed in frequency_seed:
    file_index = file_index + 1
    generated = generate_text(model, start_string=seed)
    file_name = "generated_202005" + str(file_index)  + ".pickle"
    pickle_out = open(file_name, "wb")
    pickle.dump(generated, pickle_out)
    pickle_out.close()
    print(str(file_index) + " is done")

1 is done
2 is done
3 is done
4 is done
5 is done
6 is done
7 is done
8 is done
9 is done
10 is done
